In [4]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import numpy_financial as npf

@dataclass
class ModelInputs:
    price_machine: float = 1000000  # Cost of the machine
    loan_life: int = 5              # Life of the loan in years
    initial_default: float = 0.2    # Initial default probability
    default_decay: float = 0.9      # Annual decay in default probability
    final_default: float = 0.4      # Default probability in the final year
    recovery_rate: float = 0.4      # Recovery rate in case of default
    interest: float = 0.2           # Annual interest rate
    num_iterations: int = 1000      # Number of iterations for the simulation

# Creating an instance of ModelInputs with the default values
model_data = ModelInputs()

def calculate_irr(model_inputs):
    irr_results = []
    loan_amount = model_inputs.price_machine
    recovery_rate = model_inputs.recovery_rate

    # Set up the cash flows based on default probabilities and recovery rates
    for rate in [0.3, 0.35, 0.4]:
        for loan_life in [5, 10, 20]:  # Loan life options from the provided solution
            for initial_default_prob in [0.1, 0.2, 0.3]:  # Initial default probabilities
                irrs = []
                for _ in range(model_inputs.num_iterations):
                    cash_flows = [-loan_amount]  # Initial loan amount (negative outflow)
                    default_occurred = False
                    default_year = None

                    for year in range(1, loan_life + 2):  # Extend to accommodate bankruptcy process
                        # If a default has occurred, check if it's time to recover funds
                        if default_occurred and (year - default_year == 3):
                            recovery_amount = recovery_rate * loan_amount
                            cash_flows.append(recovery_amount)
                            continue
                        # If the year is during the bankruptcy process, no cash flows are collected
                        elif default_occurred and (year - default_year <= 2):
                            cash_flows.append(0)
                            continue

                        # Simulate whether a default happens this year
                        if not default_occurred and np.random.rand() < initial_default_prob:
                            default_occurred = True
                            default_year = year
                            cash_flows.append(0)  # Default, no cash flow this year
                        elif not default_occurred:
                            cash_flows.append(loan_amount * rate)  # Interest payment
                        
                    # If no default occurred during the loan life, add the final repayment
                    if not default_occurred:
                        cash_flows.append(loan_amount)

                    irr = npf.irr(cash_flows)
                    if not np.isnan(irr):  # Filtering out invalid IRR calculations
                        irrs.append(irr)

                average_irr = np.mean(irrs) if irrs else None  # Average IRR if there are valid calculations
                irr_results.append((rate, loan_life, initial_default_prob, average_irr))

    # Creating a DataFrame from the results
    irr_df = pd.DataFrame(irr_results, columns=['Interest Rate', 'Loan Life', 'Initial Default Probability', 'IRR'])
    return irr_df


irr_df = calculate_irr(model_data)
print(irr_df)  # Displaying the first few rows of the DataFrame for verification

    Interest Rate  Loan Life  Initial Default Probability       IRR
0            0.30          5                          0.1  0.137354
1            0.30          5                          0.2  0.034422
2            0.30          5                          0.3 -0.047742
3            0.30         10                          0.1  0.141123
4            0.30         10                          0.2  0.041537
5            0.30         10                          0.3 -0.020931
6            0.30         20                          0.1  0.139551
7            0.30         20                          0.2  0.047203
8            0.30         20                          0.3 -0.031322
9            0.35          5                          0.1  0.166861
10           0.35          5                          0.2  0.063810
11           0.35          5                          0.3 -0.001979
12           0.35         10                          0.1  0.178865
13           0.35         10                    